In [1]:
import os
import sys
print(sys.path)
import BioSimSpace as BSS

['/opt/miniforge/envs/openbiosim/lib/python39.zip', '/opt/miniforge/envs/openbiosim/lib/python3.9', '/opt/miniforge/envs/openbiosim/lib/python3.9/lib-dynload', '', '/home/purohit.52@pharmacy.ohio-state.edu/.local/lib/python3.9/site-packages', '/opt/miniforge/envs/openbiosim/lib/python3.9/site-packages']


INFO:rdkit:Enabling RDKit 2023.09.6 jupyter extensions
INFO:numexpr.utils:Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [2]:
atp = BSS.IO.readMolecules(["inputs/ligands/atp_maegher.prm7","inputs/ligands/atp_maegher.rst7"])[0]   
adp = BSS.IO.readMolecules(["inputs/ligands/adp_maegher.prm7","inputs/ligands/adp_maegher.rst7"])[0]   


In [3]:
mapping = BSS.Align.matchAtoms(atp, adp)
# Mapping is a dictionary mapping atom indices in ethane to those in methanol.
print(mapping)
inv_mapping = {v: k for k, v in mapping.items()}
adp_aligned = BSS.Align.rmsdAlign(adp, atp, inv_mapping)

{4: 3, 5: 1, 6: 0, 7: 2, 8: 4, 9: 5, 10: 7, 11: 6, 12: 8, 13: 9, 14: 10, 15: 11, 16: 12, 17: 13, 18: 14, 19: 15, 20: 16, 21: 17, 22: 18, 23: 19, 24: 20, 25: 21, 26: 22, 27: 23, 28: 25, 29: 24, 30: 26, 31: 27, 32: 28, 33: 29, 34: 30, 35: 31, 36: 32, 37: 33, 38: 34, 39: 35, 40: 36, 41: 37, 42: 38}


In [5]:
merged = BSS.Align.merge(atp, adp_aligned, mapping)
complex_sol = BSS.Solvent.tip3p(molecule=merged, box=3*[30*BSS.Units.Length.angstrom], ion_conc=0.15, is_neutral=True)
protocol = BSS.Protocol.FreeEnergy(runtime=0.01*BSS.Units.Time.nanosecond, num_lam=3)
fep_bound = BSS.FreeEnergy.Relative(complex_sol, protocol, engine="GROMACS", ignore_warnings=True, work_dir="atp_adp")
fep_bound.run()  
fep_bound.wait()
print("done")

done
